## 1. Ham Deng

### Variables:

x₁: Number of times to run "Good morning" elderly messaging campaign
x₂: Number of times to run HamTok short videos
x₃: Number of times to run Hambook posts
x₄: Number of times to run Hamtube documentaries
x₅: Number of times to run Y hashtag campaign
x₆: Number of times to run disinformation campaign

### Objective Function:
Minimize Total Cost = 1000x₁ + 10000x₂ + 10000x₃ + 25000x₄ + 50000x₅ + 600000x₆
Key Constraints:

###  Total Reach Constraint:
(1000 * x₁) + (50000 * x₂) + (20000 * x₃) + (150000 * x₄) + (50000 * x₅) + (500000 * x₆) ≥ 1,000,000

Ensures at least 1 million people are reached


###  Campaign Frequency Constraints:

x₁, x₂, x₃: Max 30 times (daily campaigns)
x₄: Max 2 times (15-day delay between runs)
x₅: Max 6 times (5-day delay between runs)
x₆: Max 15 times (2-day delay between runs)


In [7]:
from pulp import *

# Create the optimization problem
prob = LpProblem("Ham_Deng_Campaign", LpMinimize)

# Define variables
# For each campaign, how many times it will run in 30 days
x1 = LpVariable("elderly_messaging", 0, 30, LpInteger)  # Can run daily
x2 = LpVariable("hamtok", 0, 30, LpInteger)            # Can run daily
x3 = LpVariable("hambook", 0, 30, LpInteger)           # Can run daily
x4 = LpVariable("hamtube", 0, 2, LpInteger)            # Max 2 times in 30 days (15 day delay)
x5 = LpVariable("y_hashtag", 0, 6, LpInteger)          # Max 6 times in 30 days (5 day delay)
x6 = LpVariable("disinfo", 0, 15, LpInteger)           # Max 15 times in 30 days (2 day delay)

# Objective function: Minimize total cost
prob += 1000*x1 + 10000*x2 + 10000*x3 + 25000*x4 + 50000*x5 + 600000*x6

# Constraint: Reach at least 1 million people
prob += 1000*x1 + 50000*x2 + 20000*x3 + 150000*x4 + 50000*x5 + 500000*x6 >= 1000000

# Solve the problem
prob.solve()

# Print results
print("Status:", LpStatus[prob.status])
print("\nOptimal Campaign Plan:")
for v in prob.variables():
    if v.varValue > 0:  # Only print campaigns that should be run
        print(f"{v.name}: {int(v.varValue)} times")

print(f"\nTotal Cost: ${int(value(prob.objective)):,}")

Status: Optimal

Optimal Campaign Plan:
hamtok: 14 times
hamtube: 2 times

Total Cost: $190,000


## 2. The Buff Boss 

### Variables:

Multiple decision variables of the form:
vars_dict[f"{ingredient}_{shopping_day}_{consumption_day}"]
Each variable represents the number of units of a specific ingredient bought on a shopping day and consumed on a particular day

### Objective Function:
Minimize Total Cost = Sum of (ingredient price * number of units purchased)
Key Constraints:

### Daily Protein Constraint (for each day):
Sum of (protein per unit * units purchased) ≥ 200g
Daily Carbohydrate Constraint (for each day):
Sum of (carbohydrates per unit * units purchased) ≥ 150g
Shelf Life Constraints:

Chicken (1-day shelf life)
Pork (3-day shelf life)
Jasmine rice (3-day shelf life)
White rice (2-day shelf life)


### Shopping Day Constraints:

Can only shop on Monday and Wednesday

In [8]:
from pulp import *

# Create the optimization problem
prob = LpProblem("Boss_Meal_Planning", LpMinimize)

# Define ingredients
ingredients = {
    'Chicken': {'price': 30, 'shelf_life': 1, 'protein': 24, 'carbs': 0},
    'Pork': {'price': 35, 'shelf_life': 3, 'protein': 21, 'carbs': 0},
    'Jasmine_rice': {'price': 40, 'shelf_life': 3, 'protein': 6, 'carbs': 80},
    'White_rice': {'price': 20, 'shelf_life': 2, 'protein': 9, 'carbs': 50}
}

# Define days
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
shopping_days = ['Monday', 'Wednesday']

# Create variables for each ingredient purchased on each shopping day
# and consumed on each possible day within shelf life
vars_dict = {}

for shop_day in shopping_days:
    shop_day_idx = days.index(shop_day)
    for ing in ingredients:
        shelf_life = ingredients[ing]['shelf_life']
        # Can only consume within shelf life days after purchase
        possible_consumption_days = days[shop_day_idx:shop_day_idx + shelf_life]
        for cons_day in possible_consumption_days:
            var_name = f"{ing}_{shop_day}_{cons_day}"
            vars_dict[var_name] = LpVariable(var_name, 0, None)

# Objective: Minimize total cost
prob += lpSum([
    vars_dict[f"{ing}_{shop_day}_{cons_day}"] * ingredients[ing]['price']
    for shop_day in shopping_days
    for ing in ingredients
    for cons_day in days[days.index(shop_day):days.index(shop_day) + ingredients[ing]['shelf_life']]
    if days.index(cons_day) < len(days)  # Ensure we don't go beyond Friday
])

# Constraints for daily nutrient requirements
for day in days:
    # Protein requirement (≥ 200g)
    prob += lpSum([
        vars_dict[f"{ing}_{shop_day}_{day}"] * ingredients[ing]['protein']
        for shop_day in shopping_days
        for ing in ingredients
        if days.index(day) >= days.index(shop_day) and  # Can't consume before purchase
        days.index(day) < days.index(shop_day) + ingredients[ing]['shelf_life'] and  # Must consume within shelf life
        days.index(day) < len(days)  # Must consume by Friday
    ]) >= 200

    # Carbohydrate requirement (≥ 150g)
    prob += lpSum([
        vars_dict[f"{ing}_{shop_day}_{day}"] * ingredients[ing]['carbs']
        for shop_day in shopping_days
        for ing in ingredients
        if days.index(day) >= days.index(shop_day) and
        days.index(day) < days.index(shop_day) + ingredients[ing]['shelf_life'] and
        days.index(day) < len(days)
    ]) >= 150

# Solve the problem
prob.solve()

# Print results
print(f"Status: {LpStatus[prob.status]}")
print(f"\nTotal Cost: {value(prob.objective):.2f} THB")

# Print detailed purchase and consumption plan
for shop_day in shopping_days:
    print(f"\nShopping on {shop_day}:")
    for ing in ingredients:
        shop_day_idx = days.index(shop_day)
        shelf_life = ingredients[ing]['shelf_life']
        possible_consumption_days = days[shop_day_idx:shop_day_idx + shelf_life]
        for cons_day in possible_consumption_days:
            var_name = f"{ing}_{shop_day}_{cons_day}"
            if var_name in vars_dict and vars_dict[var_name].varValue > 0.01:  # Small threshold to handle floating-point
                print(f"  Buy {vars_dict[var_name].varValue:.2f} units of {ing} to consume on {cons_day}")

# Print daily nutrient totals
for day in days:
    protein_total = sum([
        vars_dict[f"{ing}_{shop_day}_{day}"].varValue * ingredients[ing]['protein']
        for shop_day in shopping_days
        for ing in ingredients
        if f"{ing}_{shop_day}_{day}" in vars_dict
    ])
    carbs_total = sum([
        vars_dict[f"{ing}_{shop_day}_{day}"].varValue * ingredients[ing]['carbs']
        for shop_day in shopping_days
        for ing in ingredients
        if f"{ing}_{shop_day}_{day}" in vars_dict
    ])
    print(f"\n{day} nutrients:")
    print(f"  Protein: {protein_total:.1f}g")
    print(f"  Carbs: {carbs_total:.1f}g")

Status: Optimal

Total Cost: 1638.75 THB

Shopping on Monday:
  Buy 7.21 units of Chicken to consume on Monday
  Buy 8.24 units of Pork to consume on Tuesday
  Buy 3.00 units of White_rice to consume on Monday
  Buy 3.00 units of White_rice to consume on Tuesday

Shopping on Wednesday:
  Buy 7.21 units of Chicken to consume on Wednesday
  Buy 8.24 units of Pork to consume on Thursday
  Buy 8.99 units of Pork to consume on Friday
  Buy 1.88 units of Jasmine_rice to consume on Friday
  Buy 3.00 units of White_rice to consume on Wednesday
  Buy 3.00 units of White_rice to consume on Thursday

Monday nutrients:
  Protein: 200.0g
  Carbs: 150.0g

Tuesday nutrients:
  Protein: 200.0g
  Carbs: 150.0g

Wednesday nutrients:
  Protein: 200.0g
  Carbs: 150.0g

Thursday nutrients:
  Protein: 200.0g
  Carbs: 150.0g

Friday nutrients:
  Protein: 200.0g
  Carbs: 150.0g


## 3. Deadline Driven Development

### Variables:

Binary variables x[i,d]:

i = task number (1-7)
d = day number (1-14)
x[i,d] = 1 if task i is done on day d, 0 otherwise


Binary variables day_used[d]:

Indicates whether a specific day is used for work



### Objective Function:
Minimize Sum of day_used[d]

Goal is to minimize the number of days used to complete all tasks

### Key Constraints:

Task Completion Constraint:
For each task i, Sum of x[i,d] across all days = 1
(Each task must be done exactly once)
Daily Work Limit Constraint:
For each day d, Sum of (task hours * x[i,d]) ≤ 10
(Cannot work more than 10 hours per day)
Specific Task Constraints:

Task 2 cannot be on the same day as Task 1
Task 4 must be on the same day as Task 3
Task 7 can only be done after Tasks 5 and 6

In [9]:
from pulp import *

# Create the optimization problem (minimize number of days)
prob = LpProblem("Snoozer_Task_Scheduling", LpMinimize)

# Parameters
num_days = 14
num_tasks = 7
task_hours = {1: 6, 2: 4, 3: 5, 4: 3, 5: 6, 6: 2, 7: 4}

# Decision Variables
# x[i][d] = 1 if task i is done on day d, 0 otherwise
x = LpVariable.dicts("task", 
                     ((i, d) for i in range(1, num_tasks + 1) 
                              for d in range(1, num_days + 1)),
                     cat='Binary')

# Variable to track which days are used
day_used = LpVariable.dicts("day_used",
                           range(1, num_days + 1),
                           cat='Binary')

# Objective: Minimize number of days used
prob += lpSum(day_used[d] for d in range(1, num_days + 1))

# Constraints

# 1. Each task must be done exactly once
for i in range(1, num_tasks + 1):
    prob += lpSum(x[i,d] for d in range(1, num_days + 1)) == 1

# 2. Daily work limit of 10 hours
for d in range(1, num_days + 1):
    prob += lpSum(task_hours[i] * x[i,d] for i in range(1, num_tasks + 1)) <= 10

# 3. Link day_used variables with task assignments
for d in range(1, num_days + 1):
    for i in range(1, num_tasks + 1):
        prob += x[i,d] <= day_used[d]

# 4. Task 2 cannot be done on the same day as task 1
for d in range(1, num_days + 1):
    prob += x[1,d] + x[2,d] <= 1

# 5. Task 4 has to be done on the same day as task 3
for d in range(1, num_days + 1):
    prob += x[3,d] == x[4,d]

# 6. Task 7 can only be done after tasks 5 and 6
for d in range(1, num_days + 1):
    # Sum of days where task 7 is done multiplied by their day number
    task7_day = lpSum(d * x[7,d] for d in range(1, num_days + 1))
    # Sum of days where task 5 is done multiplied by their day number
    task5_day = lpSum(d * x[5,d] for d in range(1, num_days + 1))
    # Sum of days where task 6 is done multiplied by their day number
    task6_day = lpSum(d * x[6,d] for d in range(1, num_days + 1))
    
    # Task 7's day must be greater than both task 5 and 6's days
    prob += task7_day >= task5_day
    prob += task7_day >= task6_day

# Solve the problem
prob.solve()

# Print results
print(f"Status: {LpStatus[prob.status]}")
print(f"Minimum number of days needed: {int(value(prob.objective))}")

# Create a day-by-day schedule
schedule = {}
for d in range(1, num_days + 1):
    tasks_today = []
    hours_today = 0
    for i in range(1, num_tasks + 1):
        if value(x[i,d]) == 1:
            tasks_today.append(i)
            hours_today += task_hours[i]
    if tasks_today:
        schedule[d] = {"tasks": tasks_today, "hours": hours_today}

# Print detailed schedule
print("\nDetailed Schedule:")
for day, info in sorted(schedule.items()):
    tasks_str = ", ".join(f"Task {t}" for t in info["tasks"])
    print(f"Day {day}: {tasks_str} (Total: {info['hours']} hours)")

# Verify all constraints are met
print("\nConstraint Verification:")
print("1. Maximum 10 hours per day:", all(info["hours"] <= 10 for info in schedule.values()))
print("2. Tasks 1 and 2 not on same day:", all(not (1 in info["tasks"] and 2 in info["tasks"]) for info in schedule.values()))
print("3. Tasks 3 and 4 on same day:", any(3 in info["tasks"] and 4 in info["tasks"] for info in schedule.values()))

# Verify task 7 comes after tasks 5 and 6
task5_day = next(day for day, info in schedule.items() if 5 in info["tasks"])
task6_day = next(day for day, info in schedule.items() if 6 in info["tasks"])
task7_day = next(day for day, info in schedule.items() if 7 in info["tasks"])
print("4. Task 7 after Tasks 5 and 6:", task7_day > task5_day and task7_day > task6_day)

Status: Optimal
Minimum number of days needed: 3

Detailed Schedule:
Day 1: Task 3, Task 4, Task 6 (Total: 10 hours)
Day 4: Task 2, Task 5 (Total: 10 hours)
Day 9: Task 1, Task 7 (Total: 10 hours)

Constraint Verification:
1. Maximum 10 hours per day: True
2. Tasks 1 and 2 not on same day: True
3. Tasks 3 and 4 on same day: True
4. Task 7 after Tasks 5 and 6: True


## References

- Gemini for main code(https://claude.ai/chat/c7356632-e47e-46db-a6e7-98a1f1b85e3f)
- ChatGPT for verification (https://chatgpt.com/c/f89884a9-b370-42b7-b504-a7b0e4646bcd)
- Check optimal answers with many of my friends (Bee, Veeliw, Mark, Get)